In [1]:
from chain_logging import langsmith
from dotenv import load_dotenv

load_dotenv()

langsmith("langchain_RAG")


LangSmith 추적을 시작합니다.
[프로젝트명]
langchain_RAG


In [2]:
import os
from utils import get_embeddings
from vectorstore_manager import (
    load_vectorstore_from_FAISS,
    create_vectorstore_from_FAISS,
    create_vectorstore_from_Mongo,
    load_vectorestore_from_Mongo,
    get_connection,
)

def load_vectorestore_FAISS(vectorestore_path, pdf_path):
    embeddings = get_embeddings()

    # Load or create vectorstore
    if os.path.exists(vectorestore_path):
        print("Loading existing vectorstore...")
        loaded_vectorstore = load_vectorstore_from_FAISS(vectorestore_path, embeddings)
    else:
        print("Vectorstore not found. Creating a new one...")
        loaded_vectorstore = create_vectorstore_from_FAISS(
            pdf_path, vectorestore_path, embeddings
        )

    return loaded_vectorstore


def load_vectorestore_Mongo(pdf_path):

    collection = get_connection()
    embeddings = get_embeddings()

    # Load or create vectorstore
    if collection.collection_has_data():
        print("Loading existing vectorstore...")
        loaded_vectorstore = load_vectorestore_from_Mongo(embeddings)
    else:
        print("Vectorstore not found. Creating a new one...")
        loaded_vectorstore = create_vectorstore_from_Mongo(pdf_path, embeddings)        

    return loaded_vectorstore

### 대화 내용을 기억하는 기능을 추가

In [3]:
import os

from config import LOCAL_MODEL_PATH, MODEL_NAME, MODEL_PATH, PDF_PATH, VECTORSTORE_PATH
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from load_model import create_model, load_model
from utils import create_prompt, format_docs_with_print
from vectorstore_manager import create_conversaion_memory

# 대화를 기억할 메모리 객체 생성
memory = create_conversaion_memory("chat_history", "answer")


# 단계 5: 리트리버 생성
k = 3

loaded_vectorstore = load_vectorestore_FAISS(VECTORSTORE_PATH, PDF_PATH)
retriever = loaded_vectorstore.as_retriever(search_kwargs={"k": k})


# 단계 6: 프롬프트 생성
prompt = create_prompt()

# 단계7: 언어모델 생성
# Ollama 모델을 불러옵니다.
# llm = ChatOllama(model=MODEL_PATH, temperature=0)

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name=MODEL_PATH,  # 모델명
)

# if os.path.exists(LOCAL_MODEL_PATH):
#     print("Loading existing Local Model...")
#     llm = load_model(LOCAL_MODEL_PATH)
# else:
#     print("Local Model not found. Creating a new one...")
#     create_model(MODEL_NAME,LOCAL_MODEL_PATH)
#     llm = load_model(LOCAL_MODEL_PATH)




c:\Users\SSAFY\ssafy\자율프로젝트\S11P31S102\AI\chain\vectorstore_manager.py:103: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(


Loading existing vectorstore...


In [12]:
# 단계 8: 체인 생성
# 단계 8: 체인 생성
rag_chain = (
    {
        "context": retriever | format_docs_with_print,
        "question": RunnablePassthrough(),
        "chat_history": lambda x: memory.load_memory_variables({})["chat_history"],
    }
    | prompt    
)

In [13]:
question = "언제 퇴직금이 지급되나요?"

response = rag_chain.invoke(question)

response


=== Retrieved Documents ===

Document 1:
../kisa_pdf/2_06_급여규정(240221).pdf |  page: 12
<별표 제4호> <개정 2014.9.5., 2016. 12. 22.> <삭제 2023. 7. 5.>
<별표 제5호>
명예퇴직금
정년잔여기간 산 정 기 준
1년 이상
퇴직당시 연봉월액 × 정년잔여월수 × 50%
5년 이내인 자
5년 초과 정년잔여월수-60
퇴직당시 연봉월액 × (60+ ) × 50%
10년 이내인 자 2
10년 초과인 자 정년 잔여기간이 10년인 자의 금액과 동일한 금액
<별표 제6호> <신설 2010. 12. 29.><개정 2014. 2. 26, 2014. 7. 4., 2018. 7. 6., 2022. 12. 22.>
직무급
직무급 가 등급 나 등급 다 등급
연지급액 13,800,000원 10,200,000원 7,200,000원
직무설명 본부급을 총괄하는 직무 단급을 총괄하는 직무 팀급을 총괄하는 직무
- 13 -
--------------------------------------------------

Document 2:
../kisa_pdf/3_09_급여지급규칙(240214).pdf |  page: 9
[별지 제4호]<개정 2009. 12. 24>
퇴직금 중간정산 신청서
담 당 담 당
결 접
재 수
신청일자 신청번호
부서(팀) 성명(사번)
직 급 입사일자
주 소
정산기간 년 ~ 년
위 상기명 본인은 급여규정 제17조 및
급여지급규칙 제13조에 따라 퇴직금 중간정산을 신청합니다.
--------------------------------------------------

Document 3:
../kisa_pdf/2_06_급여규정(240221).pdf |  page: 4
을 증명하는 서류를 제출하여야 한다.
제22조(퇴직금의 특례) <삭제 2014.9.5.>
제23조(퇴직금의 지급시기) 퇴직금은 퇴직금 지급 사유가 발생한 날부터 상당한 사유가 없는 한 14일
이내에 통화로 지급함을 원

ChatPromptValue(messages=[SystemMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.\n        If you don't know the answer, just say that you don't know.\n        Use three sentences maximum and keep the answer concise. Answer in Korean.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Context: <별표 제4호> <개정 2014.9.5., 2016. 12. 22.> <삭제 2023. 7. 5.>\n<별표 제5호>\n명예퇴직금\n정년잔여기간 산 정 기 준\n1년 이상\n퇴직당시 연봉월액 × 정년잔여월수 × 50%\n5년 이내인 자\n5년 초과 정년잔여월수-60\n퇴직당시 연봉월액 × (60+ ) × 50%\n10년 이내인 자 2\n10년 초과인 자 정년 잔여기간이 10년인 자의 금액과 동일한 금액\n<별표 제6호> <신설 2010. 12. 29.><개정 2014. 2. 26, 2014. 7. 4., 2018. 7. 6., 2022. 12. 22.>\n직무급\n직무급 가 등급 나 등급 다 등급\n연지급액 13,800,000원 10,200,000원 7,200,000원\n직무설명 본부급을 총괄하는 직무 단급을 총괄하는 직무 팀급을 총괄하는 직무\n- 13 -\n\n[별지 제4호]<개정 2009. 12. 24>\n퇴직금 중간정산 신청서\n담 당 담 당\n결 접\n재 수\n신청일자 신청번호\n부서(팀) 성명(사번)\n직 급 입사일자\n주 소\n정산기간 년 ~ 년\n위 상기명 본인은 급여규정 제17조 및\n급여지급규칙 제13조에 따라 퇴직금 중간정산을 신청합니다

In [4]:
from chain_output_stream import stream_response

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
# question = "연차는 어떻게 사용할 수 있나요?"
# question = "유류비 청구는 어떻게 할 수 있나요? 제가 3박 4일로 부산에 갔다오는데 비용은 대충 어떻게 계산하죠?"
# question = "아까 물어본 질문을 다시 알려주세요"
# question = "급여 지급일은 언제인가요?"
question = "언제 퇴직금이 지급되나요?"

response = rag_chain.stream(question)

# 결과 출력
print(f"[HUMAN]\n{question}\n")
print("===" * 20)
answer = stream_response(response, return_output=True)

# 메모리에 대화 내용을 저장
memory.save_context({"question": question}, {"answer": answer})



[HUMAN]
언제 퇴직금이 지급되나요?


=== Retrieved Documents ===

Document 1:
../kisa_pdf/2_06_급여규정(240221).pdf |  page: 12
<별표 제4호> <개정 2014.9.5., 2016. 12. 22.> <삭제 2023. 7. 5.>
<별표 제5호>
명예퇴직금
정년잔여기간 산 정 기 준
1년 이상
퇴직당시 연봉월액 × 정년잔여월수 × 50%
5년 이내인 자
5년 초과 정년잔여월수-60
퇴직당시 연봉월액 × (60+ ) × 50%
10년 이내인 자 2
10년 초과인 자 정년 잔여기간이 10년인 자의 금액과 동일한 금액
<별표 제6호> <신설 2010. 12. 29.><개정 2014. 2. 26, 2014. 7. 4., 2018. 7. 6., 2022. 12. 22.>
직무급
직무급 가 등급 나 등급 다 등급
연지급액 13,800,000원 10,200,000원 7,200,000원
직무설명 본부급을 총괄하는 직무 단급을 총괄하는 직무 팀급을 총괄하는 직무
- 13 -
--------------------------------------------------

Document 2:
../kisa_pdf/3_09_급여지급규칙(240214).pdf |  page: 9
[별지 제4호]<개정 2009. 12. 24>
퇴직금 중간정산 신청서
담 당 담 당
결 접
재 수
신청일자 신청번호
부서(팀) 성명(사번)
직 급 입사일자
주 소
정산기간 년 ~ 년
위 상기명 본인은 급여규정 제17조 및
급여지급규칙 제13조에 따라 퇴직금 중간정산을 신청합니다.
--------------------------------------------------

Document 3:
../kisa_pdf/2_06_급여규정(240221).pdf |  page: 4
을 증명하는 서류를 제출하여야 한다.
제22조(퇴직금의 특례) <삭제 2014.9.5.>
제23조(퇴직금의 지급시기) 퇴직금은 퇴직금 지급 사유가 발생한 날부터 상당한 사유가